# 3D Tennis Shot Recognition through Spatial-Temporal Graph Convolutional Networks

Using the method described in [Learning Three Dimensional Tennis Shots UsingGraph Convolutional Networks](https://www.mdpi.com/1424-8220/20/21/6094) by M. Skublewska-Paszkowska, P. Powroznikand, E. Lukasi (2020).

The paper mentions recording both the player and the racket to identify shots. It tested putting data into the ST-GCN with and without fuzzying of data, where fuzzying returned more accurate results.
The input used is from the 3D skeleton videos from the [THETIS](http://thetis.image.ece.ntua.gr)(THree dimEnsional TennIs Shots human action dataset).

The ST-GCN is implemented with the [Spektral](https://graphneural.network/getting-started/) library.

Loading the dataset into graphs.

In [17]:
from spektral import data as spkdata
from os import listdir, path, mkdir
from os.path import isdir, join
from numpy import asarray, savetxt, ndarray# just add the imports that are actually needed
import numpy as np

joint_names = ['HEAD',
    'LEFT_ELBOW',
    'LEFT_FOOT',
    'LEFT_HAND',
    'LEFT_HIP',
    'LEFT_KNEE',
    'LEFT_SHOULDER',
    'NECK',
    'RIGHT_ELBOW',
    'RIGHT_FOOT',
    'RIGHT_HAND',
    'RIGHT_HIP',
    'RIGHT_KNEE',
    'RIGHT_SHOULDER',
    'TORSO']


def create_dir(dir):
    if not isdir(dir):
        mkdir(dir)


def skeleton_to_matrix(data_path:str) -> list:  # return a more specific type. this is too vague
    joint_no = 0
    matrices = tuple([[] for _ in range(15)])
    with open(data_path) as file:
        for line in file:

            if '\n' == line: # end of file
                break

            if 'FRAME' in line: # start filling in data for next frame
                joint_no = 0
                continue

            coordinates = [float(c) for c in line.split(' ')]
            matrices[joint_no].append(coordinates) # adds the coordinates for one frame
            joint_no += 1

    return asarray(matrices)





class JointDataset(spkdata.Dataset):

    def __init__(self, frame_count: int, coordinate_count: int, **kwargs):
        self.frame_count = frame_count
        self.coordinate_count = coordinate_count

        super().__init__(**kwargs)

    def download_shot_type(self, root:str, out_dir:str) -> None:
        skeleton_paths = [ _ for _ in listdir(root)]
        
        for s in skeleton_paths:

            in_path = join(root, s)
            create_dir(out_dir)

            matrix_array = skeleton_to_matrix(in_path) # this is what needs to be maintained
            s_name = s.replace('.txt', '')
            # file saving should be here
            filename = path.join(out_dir, f'graph_{s_name}')
            np.savez(
                filename, 
                HEAD = matrix_array[0],
                LEFT_ELBOW = matrix_array[1],
                LEFT_FOOT = matrix_array[2],
                LEFT_HAND = matrix_array[3],
                LEFT_HIP = matrix_array[4],
                LEFT_KNEE = matrix_array[5],
                LEFT_SHOULDER = matrix_array[6],
                NECK = matrix_array[7],
                RIGHT_ELBOW = matrix_array[8],
                RIGHT_FOOT = matrix_array[9],
                RIGHT_HAND = matrix_array[10],
                RIGHT_HIP = matrix_array[11],
                RIGHT_KNEE = matrix_array[12],
                RIGHT_SHOULDER = matrix_array[13],
                TORSO = matrix_array[14]
            ) # would like to avoid hardcoding this

    def download_expertise(self, expertise, overarching_dir='./data/THETIS_Skeletal_Joints/normal_oniFiles/') -> None:
        root = join(overarching_dir, expertise)
        create_dir(join('./data/skeleton_npz', expertise))
        for shot in [directory for directory in listdir(root) if isdir(join(root, directory))]:
            self.download_shot_type(join(root, shot), join('./data/skeleton_npz', expertise) + '/' + shot + '/')

    def download(self):

        create_dir('./data/skeleton_npz')

        self.download_expertise('ONI_AMATEURS')
        self.download_expertise('ONI_EXPERTS')

    def read(self):
        # should impl this tomorrow
        output = []
        for shot in [direc for direc in listdir('./data/skeleton_npz/ONI_AMATEURS/') if isdir(join('./data/skeleton_npz/ONI_AMATEURS/', direc))]:
            for path in [_ for _ in listdir(join('./data/skeleton_npz/ONI_AMATEURS/', shot))]:
                data = np.load(join(join('./data/skeleton_npz/ONI_AMATEURS/', shot), path))
                output.append(spkdata.Graph(
                    HEAD = data['HEAD'],
                    LEFT_ELBOW = data['LEFT_ELBOW'],
                    LEFT_FOOT = data['LEFT_FOOT'],
                    LEFT_HAND = data['LEFT_HAND'],
                    LEFT_HIP = data['LEFT_HIP'],
                    LEFT_KNEE = data['LEFT_KNEE'],
                    LEFT_SHOULDER = data['LEFT_SHOULDER'],
                    NECK = data['NECK'],
                    RIGHT_ELBOW = data['RIGHT_ELBOW'],
                    RIGHT_FOOT = data['RIGHT_FOOT'],
                    RIGHT_HAND = data['RIGHT_HAND'],
                    RIGHT_HIP = data['RIGHT_HIP'],
                    RIGHT_KNEE = data['RIGHT_KNEE'],
                    RIGHT_SHOULDER = data['RIGHT_SHOULDER'],
                    TORSO = data['TORSO']
                ))

        return output


dataset = JointDataset(15, 3)



In [9]:
from spektral import data as spkdata
from os import path
import numpy as np

test_dir = './data/test'


class TestDatset(spkdata.Dataset):

    def __init__(self, nodes, feats, **kwargs):
        self.nodes = nodes # this is just created input?
        self.feats = feats # this is also just created input?

        super().__init__(**kwargs)

    def download(self):

        for i in range(5):
            # one file for each thing
            node_features = np.random.rand(self.nodes, self.feats) # makes a regular matrix
            adj = np.random.randint(0,2, (self.nodes, self.nodes)) # makes an adjacency matrix
            labels = np.random.randint(0,2) # just a single number
            
            filename = path.join(test_dir, f'graph_{i}')
            np.savez(filename, x=node_features, a=adj, y=labels)

    def read(self):
        # We must return a list of Graph objects
        output = []

        for i in range(5):
            data = np.load(path.join(test_dir, f'graph_{i}.npz')) 
            output.append(
                spkdata.Graph(x=data['x'], a=data['a'], y=data['y'])
            )
        return output

dataset = TestDatset(3,2)
dataset
graph = dataset.read()